In [1]:
from os import path
from aicsimageio import AICSImage
from matplotlib import pyplot as plt
import basicpy
import numpy as np
from dask import array as da
from tqdm import tqdm
import os
from skimage import transform, filters, morphology
import yaml

In [2]:
import sys
sys.path.append("/mnt/showers/AxioObserver7/ImageData/Fukai/image_analysis/230203_timelapse")
from a_output_metadata import output_metadata
from b_basicpy_shading_correction import basicpy_shading_calculation, plot_shading_correction_result, correct_shading

In [3]:
basedir = "/mnt/showers/AxioObserver7/ImageData/Fukai/2023-01-27-timelapse/"
workdir = "/work/fukai/2023-01-27-timelapse/"
file_path_frames = [
    ("/mnt/showers/AxioObserver7/ImageData/Fukai/2023-01-27-timelapse/2023-02-03/230203-atRA-nolive-stage-staining.czi",0),
    ("/mnt/showers/AxioObserver7/ImageData/Fukai/2023-01-27-timelapse/2023-02-03/230203-atRA-live-staining.czi",0),
    ("/mnt/showers/AxioObserver7/ImageData/Fukai/2023-01-27-timelapse/2023-02-03/230203-atRA-nolive-staining.czi",0),
    ("/mnt/showers/AxioObserver7/ImageData/Fukai/2023-01-27-timelapse/2023-02-03/230203-control-nolive-staining.czi",0),
    ("/mnt/showers/AxioObserver7/ImageData/Fukai/2023-01-27-timelapse/2023-01-30/230130-atRA-live-2.czi/230130-atRA-live-2_AcquisitionBlock1.czi/230130-atRA-live-2_AcquisitionBlock1_pt1.czi",100)
]

In [4]:
for file_path, frame in file_path_frames:
    assert path.isfile(file_path), f"{file_path} is not a file"

In [10]:
for file_path, frame in tqdm(file_path_frames[-1:]):
    metadata_path = path.join(workdir,path.relpath(file_path,basedir)+"_metadata.yaml")
    os.makedirs(path.dirname(metadata_path),exist_ok=True)
    output_metadata(file_path,metadata_path)

100%|██████████| 1/1 [07:10<00:00, 430.03s/it]


In [11]:
file_path_frames[0][0]

'/mnt/showers/AxioObserver7/ImageData/Fukai/2023-01-27-timelapse/2023-02-03/230203-atRA-nolive-stage-staining.czi'

In [5]:
img = AICSImage(file_path_frames[0][0],reconstruct_mosaic=False)
img_data=img.get_image_dask_data("CMZYX",T=file_path_frames[0][1])

In [6]:
mosaic_positions = [list(img.get_mosaic_tile_position(i)) for i in range(img.dims.M)]

In [8]:
!pip install aicslibczi

ERROR: Could not find a version that satisfies the requirement aicslibczi (from versions: none)
ERROR: No matching distribution found for aicslibczi


In [9]:
from aicspylibczi import CziFile
with open(file_path_frames[0][0]) as f:
    czi = CziFile(f)
    bboxes = czi.get_all_mosaic_tile_bounding_boxes()
    bbox = list(bboxes.values())
mosic_positions = [(b.y, b.x) for b in bbox]

In [10]:
img.dims

<Dimensions [M: 88, T: 1, C: 4, Z: 1, Y: 2048, X: 2048]>

In [ ]:
plt.imshow(img_data[-1,0,0])

# Shading correction

In [ ]:
for file_path, frame in file_path_frames:
    metadata_path = path.join(workdir,path.relpath(file_path,basedir)+"_metadata.yaml")
    with open(metadata_path,"r") as f:
        channel_names = yaml.safe_load(f)["channel_names"]
    phase_channel = channel_names.index("Phase10x")
    
    img = AICSImage(file_path,reconstruct_mosaic=False)
    img_data = img.get_image_dask_data("CMZYX",T=frame)
    shading_fitting_results = basicpy_shading_calculation(img_data)
    fig = plot_shading_correction_result(shading_fitting_results)
    fig.show()
    img_data_corrected = correct_shading(img_data,shading_fitting_results, local_subtraction_channels=[phase_channel])
    img_data_corrected = img_data_corrected.rechunk((1,1,1,*img_data.shape[3:]))
    plt.figure(figsize=(5,5))
    plt.imshow(img_data_corrected[phase_channel,0,0])
    plt.show()

    zarr_path = path.join(workdir,path.relpath(file_path,basedir)+"_rescaled.zarr")
    os.makedirs(path.dirname(zarr_path),exist_ok=True)
    img_data_corrected.to_zarr(zarr_path)

In [5]:
from aicspylibczi import CziFile

In [6]:
input_czi = file_path_frames[3][0]
with open(input_czi) as f:
    czi = CziFile(f)
    bboxes = czi.get_all_mosaic_tile_bounding_boxes()
    bbox = list(bboxes.values())
mosic_positions = [(b.y, b.x) for b in bbox]

In [7]:
input_czi = file_path_frames[3][0]
img = AICSImage(input_czi,reconstruct_mosaic=False)

In [ ]:
mosaic_positions = [list(img.get_mosaic_tile_position(i)) 
                            for i in tqdm(range(img.dims.M))]